# Set up Colab

In [1]:
%pip install -qU torchinfo python-dotenv pytorch-ignite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 18.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Add src module
import sys

sys.path.insert(0, str("/content/drive/MyDrive/AI Talent Hub/Обработка и генерация изображений/images_hw_3"))
print(sys.path)

['/content/drive/MyDrive/AI Talent Hub/Обработка и генерация изображений/images_hw_3', '/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython']


# Set up notebook

In [79]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
import random
from multiprocessing import cpu_count
from pathlib import Path

import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms.v2 as transforms
import wandb
from dotenv import load_dotenv
from torch.utils.data import DataLoader, Subset
from torchinfo import summary
from torchvision.models import resnet18

from src.metrics import Metrics
from src.pipeline import train, test
from src.utils import Identity

In [6]:
# Supress torch dynamo warnings
import torch._dynamo

torch._dynamo.config.suppress_errors = True

In [7]:
# Load wandb key

# Local
# load_dotenv()

# Colab
from google.colab import userdata
WANDB_KEY = userdata.get("WANDB_KEY")

In [8]:
# Ensure deterministic behavior
from numpy.random import MT19937, RandomState, SeedSequence

random.seed(3)
rs = RandomState(MT19937(SeedSequence(3)))
torch.backends.cudnn.deterministic = True
torch.manual_seed(3)
torch.cuda.manual_seed_all(3)

In [81]:
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Set up data

In [10]:
transform = transforms.Compose(
    [
        transforms.ToImage(),
        transforms.ToDtype(torch.float32, scale=True),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

batch_size = 512

classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")
i2c = {i: c for i, c in enumerate(classes)}

trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

100%|██████████| 170M/170M [00:45<00:00, 3.76MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [11]:
# Make validation set
valset_ids = rs.choice(len(trainset), 5000, False)
valset = Subset(trainset, valset_ids)

trainset = Subset(trainset, np.delete(np.arange(len(trainset)), valset_ids))

In [12]:
[len(d) for d in (trainset, valset, testset)]

[45000, 5000, 10000]

In [13]:
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=cpu_count())
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=cpu_count())
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=cpu_count())

# Train

## Teacher

In [82]:
model = resnet18()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Edit model

In [15]:
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = Identity()
model.layer4 = Identity()
model.fc = nn.Linear(256, 10)

In [16]:
model = torch.compile(model.to(device))

### Analyze model

In [17]:
summary(
    model,
    input_data=torch.zeros(size=(batch_size, 3, 32, 32), dtype=torch.float32, device=device),
    mode="train",
)

Layer (type:depth-idx)                        Output Shape              Param #
OptimizedModule                               [512, 10]                 --
├─ResNet: 1-1                                 [512, 10]                 --
│    └─Conv2d: 2-1                            [512, 64, 32, 32]         1,728
│    └─BatchNorm2d: 2-2                       [512, 64, 32, 32]         128
│    └─ReLU: 2-3                              [512, 64, 32, 32]         --
│    └─Identity: 2-4                          [512, 64, 32, 32]         --
│    └─Sequential: 2-5                        [512, 64, 32, 32]         --
│    │    └─BasicBlock: 3-1                   [512, 64, 32, 32]         73,984
│    │    └─BasicBlock: 3-2                   [512, 64, 32, 32]         73,984
│    └─Sequential: 2-6                        [512, 128, 16, 16]        --
│    │    └─BasicBlock: 3-3                   [512, 128, 16, 16]        230,144
│    │    └─BasicBlock: 3-4                   [512, 128, 16, 16]        295,42

### Training

In [18]:
# Login into wandb

# Local
# wandb.login(key=os.getenv("WANDB_KEY"))

# Colab
wandb.login(key=WANDB_KEY)

wandb: Currently logged in as: deniskirbaba (deniskirbaba-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [19]:
# Define hyperparameters
config = dict(
    model_base="resnet18",
    n_layers=14,
    n_params=2.7e6,
    epochs=15,
    batch_size=batch_size,
    learning_rate=3e-4,
)

In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
loss_fn = nn.CrossEntropyLoss(reduction="sum")
metrics = Metrics(i2c, device)

In [21]:
model = train(
    model,
    optimizer,
    loss_fn,
    trainloader,
    valloader,
    metrics,
    device,
    config,
    "images-hw3",
    Path("./models"),
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

Training:   0%|          | 0/88 [00:00<?, ?it/s]

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/88 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x78b5cc08d580>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
Exception ignored in:     self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
<function _MultiProcessingDataLoaderIter.__del__ at 0x78b5cc08d580>
    Traceback (most recent call last):
if w.is_alive():  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__

 Exception ignored in:      <function _MultiProcessingDataLoaderIter.__del__ at 0x78b5cc08d580>
self._shutdown_workers()
 Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
        

Training:   0%|          | 0/88 [00:00<?, ?it/s]

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

epoch,▁▁▁▁▁▁▁▂▃▃▃▃▃▃▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████
f1_avg,▁▁▆▆▅▄▅▆▄▅▇████
f1_bird,▁▃▅▆▆▄▇▇▅▄█████
f1_car,▅▁▇▇▇▅▆█▇▇█████
f1_cat,▃▅▆▅▃▆▆▄▁▅▇▇███
f1_deer,▁▄▆▆▆▆▅▇▇▆▇▇███
f1_dog,▃▁▆▆▅▆▇▇▅▄▇████
f1_frog,▁▄▅▆▆▁█▃▃▃▇▇███
f1_horse,▁▄▅▆▆▅▆▆▄▆█▇███
f1_plane,▁▂▄▅▆▄▅▄▆▃▇▇▇██
f1_ship,▄▁▇▇█▇▇▆▇▆█████


### Testing

In [22]:
# model = torch.load("", map_location=device)  # Load best model
test(model, testloader, metrics, device, config, "images-hw3")

Test:   0%|          | 0/20 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x78b5cc08d580>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x78b5cc08d580>Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    Traceback (most recent call last):
self._shutdown_workers()  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__

    Exception ignored in:   File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
self._shutdown_workers()<function _MultiProcessingDataLoaderIter.__del__ at 0x78b5cc08d580>
      File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers

if w.is_alive():Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    
 if w.i

test_f1_avg,▁
test_f1_bird,▁
test_f1_car,▁
test_f1_cat,▁
test_f1_deer,▁
test_f1_dog,▁
test_f1_frog,▁
test_f1_horse,▁
test_f1_plane,▁
test_f1_ship,▁
test_f1_truck,▁


## Student

In [24]:
torch.cuda.empty_cache()

In [83]:
model = resnet18()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Edit model

In [84]:
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = Identity()
model.layer1[1] = Identity()
model.layer2[1] = Identity()
model.layer3[1] = Identity()
model.layer4 = Identity()
model.fc = nn.Linear(256, 10)

In [85]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Identity()
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): C

In [24]:
model = torch.compile(model.to(device))

### Analyze model

In [25]:
summary(
    model,
    input_data=torch.zeros(size=(batch_size, 3, 32, 32), dtype=torch.float32, device=device),
    mode="train",
)

Layer (type:depth-idx)                        Output Shape              Param #
OptimizedModule                               [512, 10]                 --
├─ResNet: 1-1                                 [512, 10]                 --
│    └─Conv2d: 2-1                            [512, 64, 32, 32]         1,728
│    └─BatchNorm2d: 2-2                       [512, 64, 32, 32]         128
│    └─ReLU: 2-3                              [512, 64, 32, 32]         --
│    └─Identity: 2-4                          [512, 64, 32, 32]         --
│    └─Sequential: 2-5                        [512, 64, 32, 32]         --
│    │    └─BasicBlock: 3-1                   [512, 64, 32, 32]         73,984
│    │    └─Identity: 3-2                     [512, 64, 32, 32]         --
│    └─Sequential: 2-6                        [512, 128, 16, 16]        --
│    │    └─BasicBlock: 3-3                   [512, 128, 16, 16]        230,144
│    │    └─Identity: 3-4                     [512, 128, 16, 16]        --
│    └─

### Training

In [26]:
# Login into wandb

# Local
# wandb.login(key=os.getenv("WANDB_KEY"))

# Colab
wandb.login(key=WANDB_KEY)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [27]:
# Define hyperparameters
config = dict(
    model_base="resnet18",
    n_layers=8,
    n_params=1.2e6,
    epochs=15,
    batch_size=batch_size,
    learning_rate=3e-4,
)

In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
loss_fn = nn.CrossEntropyLoss(reduction="sum")
metrics = Metrics(i2c, device)

In [29]:
model = train(
    model,
    optimizer,
    loss_fn,
    trainloader,
    valloader,
    metrics,
    device,
    config,
    "images-hw3",
    Path("./models"),
)

Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

Training:   0%|          | 0/88 [00:00<?, ?it/s]

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/88 [00:00<?, ?it/s]

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

...

Training:   0%|          | 0/88 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7efe48329580>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7efe48329580>^
^Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
^^    ^^
AssertionError: self._shutdown_workers()can only test a child process

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7efe48329580

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7efe48329580>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
Exception ignored in:     <function _MultiProcessingDataLoaderIter.__del__ at 0x7efe48329580>if w.is_alive():Exception ignored in: 

<function _MultiProcessingDataLoaderIter.__del__ at 0x7efe48329580>Traceback (most recent call last):

Traceback (most recent call last):
    File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
   File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
        self._shutdown_workers()  
self._shutdown_workers()  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587,

epoch,▁▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▃▃▄▄▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇████
f1_avg,▁▃▅▅▆▇▇▇▇▇▇█▇▇▆
f1_bird,▁▂▄▄▅▇▆▆▇▇▇█▇▆▇
f1_car,▁▅▅▅▇▄▇▇▆█▇█▇▇▆
f1_cat,▁▅▆▅▇▇▇▆▇▇▅█▃█▂
f1_deer,▁▂▂▄▅█▇▇█▅▆█▇▅▆
f1_dog,▂▁▃▅▃▇▄█▇▁▆██▇▇
f1_frog,▁▃▆▆▆▇▆▇▇▇▇█▇█▇
f1_horse,▁▃▅▇▁▇▇█▇█▆███▇
f1_plane,▁▃▅▄▆▇▅▇▇█▇█▇▇▇
f1_ship,▁▂▅▅▇▇▅▇██▇▆█▇█


### Testing

In [30]:
model = torch.load("/content/models/resnet18_8_15_11.pt", map_location=device)  # Load best model
test(model, testloader, metrics, device, config, "images-hw3")

<ipython-input-30-2bc35dc5dffe>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/content/models/resnet18_8_15_11.pt", map_location=device)  # Load best 

Test:   0%|          | 0/20 [00:00<?, ?it/s]

test_f1_avg,▁
test_f1_bird,▁
test_f1_car,▁
test_f1_cat,▁
test_f1_deer,▁
test_f1_dog,▁
test_f1_frog,▁
test_f1_horse,▁
test_f1_plane,▁
test_f1_ship,▁
test_f1_truck,▁
